#### Here I am calculating virtual stock. We identify stock to be virtual if stock was unchanged during two weeks with zero sales. Also here lost sales are calculated as an average sales per outlet w/o vitrual stock times number of outlets with virtual stock

In [2]:
import os
os.chdir(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data")
import numpy as np
import pandas as pd
#!pip install openpyxl

In [3]:
#product matrix
matrix = pd.read_csv (r'C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\SO_data (4).csv')
matrix = matrix[['plu_code', 'in_matrix', 'address_text']]
matrix = matrix.drop_duplicates()


C:\Users\maiguann\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
#directory (external outlet code)
na_code = pd.read_excel (r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\NA code.xlsx", sheet_name='Sales')
na_code = na_code.iloc[6: , :]
na_code=na_code.rename(columns=na_code.iloc[0]).drop(na_code.index[0])

In [5]:
#directory (products' prices)
current_prices = pd.read_excel(r'C:\Users\maiguann\OneDrive - Mars Inc\Documents\справочник\current_prices.xlsx', sheet_name = 'E2E X5')
current_prices = current_prices.iloc[3: , :]
current_prices=current_prices.rename(columns=current_prices.iloc[0]).drop(current_prices.index[0])

In [6]:
current_prices['price'] = current_prices['Outlet Stock, RUR'].divide(current_prices['Outlet Stock, Qty']+.000000000000000000000000001)
current_prices['price']=current_prices['price'].replace([np.inf,-np.inf], np.nan)
current_prices['price']=current_prices['price'].replace(0,np.nan)

In [7]:
current_prices['Product Code']=current_prices['Product Code'].astype(int)

In [8]:
old_prices = pd.read_excel(r'C:\Users\maiguann\OneDrive - Mars Inc\Documents\справочник\ZREP-PLU-price.xlsx', sheet_name = 'ZREP-PLU-Price')
current_prices = pd.merge(current_prices, old_prices, left_on = 'Product Code', right_on = 'PLU', how='outer')

In [9]:
current_prices['price'].fillna(current_prices['BASE PRICE, PC'], inplace=True)
current_prices['Product Code'].fillna(current_prices['PLU'], inplace=True)
current_prices = current_prices[['Product Code','price']]
current_prices=current_prices.drop_duplicates(subset =['Product Code'] )
current_prices = current_prices.dropna(subset = ['price'])

In [10]:
current_prices[current_prices['price']>700]
#Extraordinary prices

,Product Code,price
237,3699502.0,3.869790e+03
238,3699503.0,1.548000e+28


## Uploading new data (stock and sales) and saving it as distinct csv files

In [ ]:
#Uploding new data (stock and sales) and saving it as distinct csv files
data = pd.read_csv(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data_csv\data_2609.csv")

In [ ]:
sales_new = data[data['Measure Names'] == 'sale_pcs']
stock_new = data[data['Measure Names'] == 'stock_pcs']
sales_new = sales_new[['address_text', 'assort_level', 'brand_name', 'day_dt', 'macro_name', 'plu_code', 'plu_name','store_dc_code', 'Measure Values']]
stock_new = stock_new[['address_text', 'assort_level', 'brand_name', 'day_dt', 'macro_name', 'plu_code', 'plu_name', 'store_dc_code', 'Measure Values']]

In [ ]:
stock_new = stock_new.pivot_table(index =['macro_name', 'address_text', 'plu_code', 'plu_name', 'store_dc_code'],columns='day_dt',values='Measure Values').reset_index() 
sales_new = sales_new.pivot_table(index =['macro_name', 'address_text', 'plu_code', 'plu_name', 'store_dc_code'],columns='day_dt',values='Measure Values').reset_index()

In [ ]:
stock_new.to_csv(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data_csv\stock MW 2609.csv")
sales_new.to_csv(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data_csv\sales MW 2609.csv")

## Creating datasets with two-weeks sales and stock

In [12]:
sales_1 = pd.read_csv (r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data_csv\sales MW 1909.csv")
sales_1.drop('Unnamed: 0', inplace=True, axis=1)
sales_2 = pd.read_csv (r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data_csv\sales MW 2609.csv")
sales_2.drop('Unnamed: 0', inplace=True, axis=1)
sales_two_weeks = pd.merge(sales_1, sales_2)

C:\Users\maiguann\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
sales_two_weeks=pd.merge(sales_two_weeks, matrix,on = ('plu_code', 'address_text'), how = 'left')
sales_two_weeks = sales_two_weeks[sales_two_weeks['in_matrix']==1]
sales_two_weeks

,werks,address_text,macro_name,assort_level,status,plu_name,plu_code,2021-09-13 00:00:00,2021-09-14 00:00:00,2021-09-15 00:00:00,...,2021-09-19 00:00:00,brand_name,2021-09-20 00:00:00,2021-09-21 00:00:00,2021-09-22 00:00:00,2021-09-23 00:00:00,2021-09-24 00:00:00,2021-09-25 00:00:00,2021-09-26 00:00:00,in_matrix
2,30A0,"129323, Москва г, Седова ул, 13А",Москва,5.0,Объект открыт,BOUNTY Конф.неж.мяк.кок.покр.мол.шок.1кг,78005190,1.0,0.0,0.0,...,1.0,BOUNTY,0.0,NaN,2.0,0.0,NaN,1.0,NaN,1.0
3,30A0,"129323, Москва г, Седова ул, 13А",Москва,5.0,Объект открыт,BOUNTY Конф.РАЙСКИЙ АНАНАС 1кг,3670735,NaN,NaN,NaN,...,0.0,BOUNTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,30A0,"129323, Москва г, Седова ул, 13А",Москва,5.0,Объект открыт,"ECLIPSE Жев.рез.ЛЕДЯНАЯ ВИШНЯ 13,6г",2111640,NaN,1.0,2.0,...,NaN,ECLIPSE,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0
7,30A0,"129323, Москва г, Седова ул, 13А",Москва,5.0,Объект открыт,M&MS Драже КРИСПИ хр.шар.мол.шок.гл.110г,4043863,NaN,NaN,NaN,...,NaN,M&M,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
8,30A0,"129323, Москва г, Седова ул, 13А",Москва,5.0,Объект открыт,M&MS Драже с ар/мол.шок.пок.хр.глаз.80г,4162277,NaN,NaN,NaN,...,2.0,M&M,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969520,Y998,"Московская обл, Ногинский р-н, Старая Купавна ...",Москва,15.0,Объект открыт,Батончик MILKY WAY шоколадный 26г,5164,NaN,1.0,4.0,...,NaN,MILKY WAY,6.0,4.0,NaN,4.0,NaN,2.0,NaN,1.0
969521,Y998,"Московская обл, Ногинский р-н, Старая Купавна ...",Москва,15.0,Объект открыт,Батончик TWIX шоколадный 55г,7650,1.0,1.0,1.0,...,NaN,TWIX,2.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0
969522,Y998,"Московская обл, Ногинский р-н, Старая Купавна ...",Москва,15.0,Объект открыт,Ж.р.ORB.REFRESH.б.с.вк.тр.фрТРОП.МИКС16г,4111967,2.0,NaN,NaN,...,NaN,ORBIT,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.0
969523,Y998,"Московская обл, Ногинский р-н, Старая Купавна ...",Москва,15.0,Объект открыт,Конфета BOUNTY с мяк.кокос.в мол.шок.55г,3663,NaN,1.0,NaN,...,NaN,BOUNTY,NaN,NaN,NaN,1.0,2.0,1.0,NaN,1.0


In [14]:
sales_two_weeks[sales_two_weeks['in_matrix']!=1]
#checkinf if merging is correct

,werks,address_text,macro_name,assort_level,status,plu_name,plu_code,2021-09-13 00:00:00,2021-09-14 00:00:00,2021-09-15 00:00:00,...,2021-09-19 00:00:00,brand_name,2021-09-20 00:00:00,2021-09-21 00:00:00,2021-09-22 00:00:00,2021-09-23 00:00:00,2021-09-24 00:00:00,2021-09-25 00:00:00,2021-09-26 00:00:00,in_matrix


In [15]:
stock_1 = pd.read_csv (r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data_csv\stock MW 1909.csv")
stock_1.drop('Unnamed: 0', inplace=True, axis=1)
stock_2 = pd.read_csv (r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data_csv\stock MW 2609.csv")
stock_2.drop('Unnamed: 0', inplace=True, axis=1)
stock_two_weeks = pd.merge(stock_1, stock_2)

C:\Users\maiguann\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
stock_two_weeks=pd.merge(stock_two_weeks, matrix,on = ('plu_code', 'address_text'), how = 'left')
stock_two_weeks = stock_two_weeks[stock_two_weeks['in_matrix']==1]


## Data on sales

In [17]:
data_sales = [col for col in sales_two_weeks if col.startswith('2021')]
data_stock = [col for col in stock_two_weeks if col.startswith('2021')]

In [18]:
sales_two_weeks ['min_sales']= sales_two_weeks[data_sales].min(axis=1)

In [19]:
sales_two_weeks ['max_sales']= sales_two_weeks[data_stock].max(axis =1)

In [20]:
sales_two_weeks['max-min'] = sales_two_weeks['max_sales']-sales_two_weeks['min_sales']

In [21]:
#only zero sales for calculating virtual stock
sales_two_weeks_zeroes = sales_two_weeks[sales_two_weeks['max_sales'].isnull()]
sales_two_weeks_zeroes = sales_two_weeks_zeroes[["werks", "macro_name", "plu_code", "address_text", "assort_level"]]

In [22]:
#nonzero sales for calculating average sales per outlet
sales_two_weeks_nonzeroes = sales_two_weeks[sales_two_weeks['max_sales']!=0]
sales_two_weeks_nonzeroes = sales_two_weeks_nonzeroes[sales_two_weeks_nonzeroes['max_sales'].notnull()]
data_sales_nonzero = [col for col in sales_two_weeks_nonzeroes if col.startswith('2021')]

In [23]:
sales_two_weeks_nonzeroes ['2_weeks_sales']= sales_two_weeks_nonzeroes[data_sales_nonzero].sum(axis =1)

In [24]:
sales_two_weeks_nonzeroes = sales_two_weeks_nonzeroes.groupby(["macro_name", "plu_code", "plu_name", "assort_level"]).agg({'werks':'size','2_weeks_sales':'sum'}).reset_index()

In [25]:
sales_two_weeks_nonzeroes['2_weeks_sales_per_outlet'] = sales_two_weeks_nonzeroes['2_weeks_sales']/sales_two_weeks_nonzeroes['werks']

## Data on stock

In [26]:
stock_two_weeks ['min_stock']= stock_two_weeks[data_stock].min(axis =1)

stock_two_weeks ['max_stock']= stock_two_weeks[data_stock].max(axis =1)

stock_two_weeks['max-min'] = stock_two_weeks['max_stock']-stock_two_weeks['min_stock']

In [27]:
stock_two_weeks = stock_two_weeks[stock_two_weeks['max_stock']!=0]
stock_two_weeks = stock_two_weeks[stock_two_weeks['max-min']==0]

In [28]:
stock_two_weeks = pd.merge(sales_two_weeks_zeroes, stock_two_weeks, how='inner')

#### Filtering out irrelevant sku

In [29]:
stock_two_weeks = stock_two_weeks[stock_two_weeks['plu_name']!="M&Ms Драж.MAXI с молочным шоколадом 70г"]
stock_two_weeks = stock_two_weeks[stock_two_weeks['plu_name']!="M&Ms Драже MAXI с арахисом 70г"]
stock_two_weeks = stock_two_weeks[stock_two_weeks['plu_name']!="M&MS Драже КРИСПИ хр.шар.мол.шок.гл.110г"]
stock_two_weeks = stock_two_weeks[stock_two_weeks['plu_name']!="M&Ms Драже с арахисом 130г"] 
stock_two_weeks = stock_two_weeks[stock_two_weeks['plu_name']!="M&Ms Драже шоколадный 130г"] 
stock_two_weeks = stock_two_weeks[stock_two_weeks['plu_name']!="SNICKERS SUP.Батончик шок.ж.ар/кар/н.95г"]

In [30]:
#Join External codes
stock_two_weeks = pd.merge(stock_two_weeks, na_code, left_on = 'werks', right_on = 'Outlet NA External Code', how = 'left')
#Выгрузим список магазинов с виртуальным стоком
stock_two_weeks.to_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\history\list_of_virtual_stock_2609.xlsx")

## Calculating lost sales

In [31]:
#current_prices['Product Code']=current_prices['Product Code'].astype(int)
stock_two_weeks['plu_code']=stock_two_weeks['plu_code'].astype(int)

In [32]:
stock_two_weeks=stock_two_weeks.dropna(subset = ['assort_level'])

In [33]:
sales_two_weeks_nonzeroes['plu_code']=sales_two_weeks_nonzeroes['plu_code'].astype(int)
sales_two_weeks_nonzeroes['assort_level']=sales_two_weeks_nonzeroes['assort_level'].astype(int)
stock_two_weeks['assort_level']=stock_two_weeks['assort_level'].astype(int)

In [34]:
#group stock dataset to macro
stock_two_weeks = stock_two_weeks.groupby(["macro_name", "plu_code", "plu_name", "assort_level"]).agg({'werks':'size','max_stock':'sum'}).reset_index()


In [35]:
# Join prices
stock_two_weeks = pd.merge(stock_two_weeks, current_prices, left_on = ['plu_code'], right_on = ['Product Code'], how = 'left')

In [36]:
stock_two_weeks['price'].isna().sum()

244

In [37]:
stock_two_weeks = pd.merge(stock_two_weeks, sales_two_weeks_nonzeroes, on = ("macro_name", "plu_code", "assort_level"), how = 'left')


In [38]:
stock_two_weeks['lost_sales_rub'] = stock_two_weeks['werks_x']*stock_two_weeks['price']*stock_two_weeks['2_weeks_sales_per_outlet']

In [39]:
stock_two_weeks.to_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\history\lost_sales_0310.xlsx")

## In dynamics (lost sales)

#### To see how lost sales are changing

In [40]:
stock_2 = pd.read_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\history\lost_sales_2609.xlsx")
stock_3 = pd.read_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\history\lost_sales_1909.xlsx")
stock_1 = pd.read_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\history\lost_sales_0310.xlsx")

In [41]:
stock_1 = stock_1 [['macro_name', 'plu_code', 'plu_name_x', 'plu_name_y','assort_level','lost_sales_rub']]
stock_2 = stock_2 [['macro_name', 'plu_code', 'plu_name_x', 'plu_name_y','assort_level','lost_sales_rub']]   
stock_3 = stock_3 [['macro_name', 'plu_code', 'plu_name_x', 'plu_name_y','assort_level','lost_sales_rub']]


In [42]:
stock_21 = pd.merge(stock_2, stock_1, on = ('macro_name', 'plu_code', 'plu_name_x', 'plu_name_y','assort_level'), how = 'outer')


In [43]:
stock_3_weeks = pd.merge(stock_3, stock_21, on = ('macro_name', 'plu_code', 'plu_name_x', 'plu_name_y','assort_level'), how = 'outer')


In [44]:
stock_3_weeks.to_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\lost_sales_3_weeks.xlsx")

## In dynamics (list of outlets)

In [45]:
stock_2 = pd.read_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\history\list_of_virtual_stock_2609.xlsx")
stock_3 = pd.read_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\history\list_of_virtual_stock_1909.xlsx")
stock_1 = pd.read_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\history\list_of_virtual_stock_0310.xlsx")

In [46]:
stock_1 = stock_1.rename(columns = {'Outlet NA SWE Code_y':'Outlet NA SWE Code'})

In [47]:
stock_1 = stock_1 [['werks', 'macro_name', 'plu_code', 'address_text','assort_level', 'plu_name','brand_name','max_stock', 'Outlet NA SWE Code']]
stock_2 = stock_2 [['werks', 'macro_name', 'plu_code', 'address_text','assort_level', 'plu_name','brand_name','max_stock', 'Outlet NA SWE Code']]   
stock_3 = stock_3 [['werks', 'macro_name', 'plu_code', 'address_text','assort_level', 'plu_name','brand_name','max_stock', 'Outlet NA SWE Code']]


In [48]:
stock_21 = pd.merge(stock_2, stock_1, on = ('werks', 'macro_name', 'plu_code', 'address_text','assort_level', 'plu_name','brand_name','Outlet NA SWE Code'), how = 'outer')


In [49]:
stock_3_weeks = pd.merge(stock_3, stock_21, on = ('werks', 'macro_name', 'plu_code', 'address_text','assort_level', 'plu_name','brand_name','Outlet NA SWE Code'), how = 'outer')

In [50]:
stock_3_weeks.to_excel(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\virtual_stock_final\list_of_outlets_3_weeks.xlsx")